In [1]:
using AutomotiveDrivingModels
using AutoViz
using AutomotiveSensors
using AutomotivePOMDPs
using Parameters
using Reel
using StaticArrays
using ProfileView

using GridInterpolations 
using POMDPs
using POMDPToolbox
using QMDP
using JLD

using Revise

In [2]:
include("../src/pedestrian_crossing/pomdp_types.jl")
include("../src/pedestrian_crossing/spaces.jl")
include("../src/pedestrian_crossing/transition.jl")
include("../src/pedestrian_crossing/observation.jl")
include("../src/pedestrian_crossing/belief.jl")

include("../src/pedestrian_crossing/frenet_pedestrian_pomdp.jl")


In [9]:
function AutomotiveDrivingModels.observe!(model::FrenetPedestrianPOMDP, scene::Scene, roadway::Roadway, egoid::Int)


    model.t_current = model.t_current + model.timestep 
    model.tick += 1
    
    ego = scene[findfirst(scene, egoid)]

    model.sensor_observations = measure(model.sensor, ego, scene, roadway, model.obstacles)


        
    ################ High Level Planner ###################################################
    if ( model.tick % model.update_tick_high_level_planner == 0 )
        
        println("--------------------------high level planner-------------------------------")
        model.sensor_observations
        for object in model.sensor_observations
          #  println(object)
            object_posF = Frenet(object.state.posG, get_lane(env.roadway, ego.state), env.roadway)

            delta_s = object_posF.s - ego.state.posF.s - ego.def.length/2
            delta_d = object_posF.t - ego.state.posF.t
            delta_theta = object_posF.ϕ - ego.state.posF.ϕ
            ped_v = object.state.v
            
            obs = SingleOCFState(0.0, ego.state.v, delta_s, delta_d, delta_theta, ped_v)
           
            
            action_pomdp = SingleOCFAction(model.a.a_lat, model.a.a_lon)
       #     b_ = update(model.updater, model.b, action_pomdp, obs)
       #     model.b = deepcopy(b_)
       #     action = action(model.policy, model.b) # policy
      #      println(action)
        
        end
    end
    
    model.risk = model.tick


    a_current = 0
    model.a = LatLonAccel(0., a_current)
    
end


In [4]:
pomdp = SingleOCFPOMDP()
updater = SingleOCFUpdater(pomdp)
b0 = initial_state_distribution(pomdp)
solver = QMDPSolver(max_iterations=50, tolerance=1e-3) 


QMDP.QMDPSolver(50, 0.001, false)

In [7]:
@requirements_info solver pomdp
# run the solver
qmdp_policy = solve(solver, pomdp, verbose=true)
JLD.save("policy.jld", "policy", qmdp_policy)

03-Okt 18:18:18:WARNING:root:Keyword args for solve(::QMDPSolver, ::POMDP) are no longer supported. For verbose output, use the verbose option in the ValueIterationSolver



INFO: POMDPs.jl requirements for solve(::QMDPSolver, ::POMDPs.POMDP) and dependencies. ([✔] = implemented correctly; [X] = missing)

For solve(::QMDPSolver, ::POMDPs.POMDP):
  [No additional requirements]
For solve(::ValueIterationSolver, ::Union{POMDPs.MDP,POMDPs.POMDP}) (in solve(::QMDPSolver, ::POMDPs.POMDP)):
  [✔] discount(::SingleOCFPOMDP)
  [✔] n_states(::SingleOCFPOMDP)
  [✔] n_actions(::SingleOCFPOMDP)
  [✔] transition(::SingleOCFPOMDP, ::SingleOCFState, ::SingleOCFAction)
  [✔] reward(::SingleOCFPOMDP, ::SingleOCFState, ::SingleOCFAction, ::SingleOCFState)
  [✔] state_index(::SingleOCFPOMDP, ::SingleOCFState)
  [✔] action_index(::SingleOCFPOMDP, ::SingleOCFAction)
  [✔] actions(::SingleOCFPOMDP, ::SingleOCFState)
  [✔] iterator(::Array)
  [✔] iterator(::Array)
  [✔] iterator(::SparseCat)
  [✔] pdf(::SparseCat, ::SingleOCFState)
For ordered_states(::Union{POMDPs.MDP,POMDPs.POMDP}) (in solve(::ValueIterationSolver, ::Union{POMDPs.MDP,POMDPs.POMDP})):
  [✔] states(::SingleOCFPO

POMDPToolbox.AlphaVectorPolicy{SingleOCFPOMDP,SingleOCFAction}(SingleOCFPOMDP
  env: AutomotivePOMDPs.CrosswalkEnv
  ego_type: AutomotiveDrivingModels.VehicleDef
  ped_type: AutomotiveDrivingModels.VehicleDef
  longitudinal_actions: Array{Float64}((5,)) [1.0, 0.0, -1.0, -2.0, -4.0]
  lateral_actions: Array{Float64}((3,)) [1.0, 0.0, -1.0]
  ΔT: Float64 0.5
  v_noise: Array{Float64}((1,)) [0.0]
  theta_noise: Array{Float64}((1,)) [0.0]
  pos_obs_noise: Float64 1.0
  vel_obs_noise: Float64 1.0
  collision_cost: Float64 -1.0
  action_cost_lon: Float64 0.0
  action_cost_lat: Float64 0.0
  goal_reward: Float64 1.0
  γ: Float64 0.95
  state_space_grid: GridInterpolations.RectangleGrid{6}
  state_space_ped: Array{SingleOCFPedState}((6250,))
  state_space: Array{SingleOCFState}((437500,))
  action_space: Array{SingleOCFAction}((15,))
, Array{Float64,1}[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0

In [5]:
qmdp_policy = load("policy.jld")["policy"];

In [10]:
params = CrosswalkParams()
params.obstacles_visible = true

obstacle_1 = ConvexPolygon([VecE2(15, -1.5), VecE2(15, -4.5), VecE2(21.5, -4.5), VecE2(21.5, -1.5)],4)
obstacle_2 = ConvexPolygon([VecE2(15, +4.5), VecE2(15, +7.5), VecE2(21.5, +7.5), VecE2(21.5, +4.5)],4)
params.obstacles = [obstacle_1, obstacle_2]
#params.obstacles = [obstacle_2]

env = CrosswalkEnv(params);




ego_v = 10.0
ped_v = 5.0/3.6
hitpoint = 50             # 50: middle, 100: left corner, 0: right corner
ped_theta = π/2


# fix values
ped_x = 25.5
ped_y_start = -2
ego_y = 0


ped_y_offset_hitpoint = VehicleDef().width * hitpoint / 100 - VehicleDef().width/2
ped_y_end = ped_y_offset_hitpoint
ped_t_collision = (ped_y_end - ped_y_start) / ped_v; 
ego_x = ped_x - ego_v * ped_t_collision - VehicleDef().length/2;


# Car definition
car_initial_state = VehicleState(VecSE2(ego_x, ego_y, 0.), env.roadway.segments[1].lanes[1], env.roadway, ego_v)
car = Vehicle(car_initial_state, VehicleDef(), 1)

# Pedestrian definition using our new Vehicle definition
ped_initial_state = VehicleState(VecSE2(ped_x,ped_y_start,ped_theta), env.crosswalk, env.roadway, ped_v)
ped = Vehicle(ped_initial_state, AutomotivePOMDPs.PEDESTRIAN_DEF, 2)

scene = Scene()
push!(scene, car)
push!(scene, ped)



cam = FitToContentCamera(0.);
timestep = 0.05


pos_noise = 0.1
vel_noise = 0.1
false_positive_rate = 0.05
false_negative_rate = 0.01
rng = MersenneTwister(1);
sensor = AutomotiveSensors.GaussianSensor(AutomotiveSensors.LinearNoise(10, pos_noise, 0.00), 
                 AutomotiveSensors.LinearNoise(10, vel_noise, 0.00), false_positive_rate, false_negative_rate, rng) 



# define a model for each entities present in the scene
models = Dict{Int, DriverModel}()

ego_id = 1
ped_id = 2
models[ego_id] = FrenetPedestrianPOMDP(a=LatLonAccel(0.0, 0.0),env=env,sensor=sensor, obstacles=env.obstacles, timestep=timestep,
    pomdp=pomdp,
    policy=qmdp_policy,
    updater=updater,
    b=b0
)
models[ped_id] = ConstantPedestrian(v_desired=ped_v, dawdling_amp=0.05) # dumb model


nticks = 80
rec = SceneRecord(nticks+1, timestep)
# execute the simulation

risk = Float64[]
sensor_observations = [Vehicle[]]

obs_callback = (ObservationCallback(risk,sensor_observations),)

tic()
simulate!(rec, scene, env.roadway, models, nticks, obs_callback)
toc()




--------------------------high level planner-------------------------------
--------------------------high level planner-------------------------------
--------------------------high level planner-------------------------------
--------------------------high level planner-------------------------------
--------------------------high level planner-------------------------------
--------------------------high level planner-------------------------------
--------------------------high level planner-------------------------------
--------------------------high level planner-------------------------------
elapsed time: 0.399765829 seconds


0.399765829

In [11]:
duration, fps, render_hist = animate_record(rec, timestep, env, sensor, sensor_observations, risk)
film = roll(render_hist, fps = fps, duration = duration)

Reel.Frames{MIME{Symbol("image/png")}}("/tmp/tmpTdJI7c", 0x0000000000000050, 20.0, nothing)